# (10) Jobs: main table

**Motivation**: Make table1 (```mc``` vs. ```exact```) job runnsers as txt file. <br>

In [1]:
# HIDE CODE


import os, sys
from IPython.display import display

# tmp & extras dir
git_dir = os.path.join(os.environ['HOME'], 'Dropbox/git')
extras_dir = os.path.join(git_dir, 'jb-vae/_extras')
fig_base_dir = os.path.join(git_dir, 'jb-vae/figs')
tmp_dir = os.path.join(git_dir, 'jb-vae/tmp')

# GitHub
sys.path.insert(0, os.path.join(git_dir, '_PoissonVAE'))
from analysis.eval import sparse_score
from figures.fighelper import *
from vae.train_vae import *

# warnings, tqdm, & style
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
from rich.jupyter import print
%matplotlib inline
set_style()

## Setup

In [2]:
from analysis.helper import job_runner_script


def _cleanup(path, host=None):
    for f in os.listdir(path):
        cond = f.endswith('.txt')
        if host is not None:
            cond = cond and host in f
        if cond:
            os.remove(pjoin(path, f))


def _name(host, gpu_i, fit_i):
    return f"{host}-cuda{gpu_i}-fit{fit_i}"


def divide_list(lst, n):
    k, m = divmod(len(lst), n)
    return [lst[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n)]

In [3]:
save_dir = 'Dropbox/git/_PoissonVAE/scripts'
save_dir = pjoin(os.environ['HOME'], save_dir)
os.makedirs(save_dir, exist_ok=True)

# delete existing job runners?
_cleanup(save_dir, None)

print(sorted(os.listdir(save_dir)))

['fit_vae.sh', 'run_sessions.sh', 'test_tqdm.py', 'test_tqdm.sh']

## Table 1 (mach)

All non-conv models

In [4]:
host = 'mach'
_cleanup(save_dir, host)

scripts_mach = collections.defaultdict(list)
tot = 0

### Part 1: mc vs. exact (main)

In [5]:
n_seeds = 5
seeds = range(1, n_seeds + 1)
args = ['--method mc', '--method exact']
datasets = ['DOVES', 'CIFAR10-PATCHES', 'MNIST']
model_types = ['poisson', 'categorical', 'gaussian', 'laplace']
archi_mach = ['lin|lin', 'mlp+b|lin']

In [6]:
combos_outer = itertools.product(
    enumerate(model_types), datasets)
for (gpu_i, m), d in combos_outer:
    combos_inner = itertools.product(
        seeds, archi_mach, args)
    for s, a, arg in combos_inner:
        kws = dict(
            device=gpu_i,
            dataset=d,
            model=m,
            archi=a,
            seed=s,
            args=arg,
        )
        scripts_mach[gpu_i].append(job_runner_script(**kws))
        tot += 1

In [7]:
print(tot)

240

### Part 2: gaussian + act fn

In [8]:
activations = ['relu', 'softplus', 'square', 'exp']

In [9]:
combos_outer = itertools.product(
    enumerate(activations), datasets)
for (gpu_i, fn), d in combos_outer:
    # arg
    arg = f"--latent_act {fn} --comment {fn}"
    if fn == 'exp':
        arg = f"{arg} --init_scale 0.0001"

    combos_inner = itertools.product(
        seeds, archi_mach)
    for s, a in combos_inner:
        kws = dict(
            device=gpu_i,
            dataset=d,
            archi=a,
            seed=s,
            args=arg,
            model='gaussian',
        )
        scripts_mach[gpu_i].append(job_runner_script(**kws))
        tot += 1

In [10]:
print(tot)

360

In [11]:
scripts_mach = dict(scripts_mach)
print({k: len(v) for k, v in scripts_mach.items()})

{0: 90, 1: 90, 2: 90, 3: 90}

### Save

In [12]:
n_fits = 6

for gpu_i, scripts in scripts_mach.items():
    scripts_divided = divide_list(scripts, n_fits)
    for fit_i, s in enumerate(scripts_divided):
        combined = ' && '.join(s)
        save_obj(
            obj=combined,
            file_name=_name(host, gpu_i, fit_i),
            save_dir=save_dir,
            mode='txt',
        )

[PROGRESS] 'mach-cuda0-fit0.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda0-fit1.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda0-fit2.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda0-fit3.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda0-fit4.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

...

[PROGRESS] 'mach-cuda3-fit1.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda3-fit2.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda3-fit3.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda3-fit4.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda3-fit5.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

### Print one to check

In [13]:
print(combined.replace('&& ', '&& \n'))

./fit_vae.sh '3' 'CIFAR10-PATCHES' 'gaussian' 'mlp+b|lin' --seed 3 --latent_act exp --comment exp --init_scale 
0.0001 && 
./fit_vae.sh '3' 'CIFAR10-PATCHES' 'gaussian' 'lin|lin' --seed 4 --latent_act exp --comment exp --init_scale 0.0001
&& 
./fit_vae.sh '3' 'CIFAR10-PATCHES' 'gaussian' 'mlp+b|lin' --seed 4 --latent_act exp --comment exp --init_scale 
0.0001 && 
./fit_vae.sh '3' 'CIFAR10-PATCHES' 'gaussian' 'lin|lin' --seed 5 --latent_act exp --comment exp --init_scale 0.0001
&& 
./fit_vae.sh '3' 'CIFAR10-PATCHES' 'gaussian' 'mlp+b|lin' --seed 5 --latent_act exp --comment exp --init_scale 
0.0001 && 
./fit_vae.sh '3' 'MNIST' 'gaussian' 'lin|lin' --seed 1 --latent_act exp --comment exp --init_scale 0.0001 && 
./fit_vae.sh '3' 'MNIST' 'gaussian' 'mlp+b|lin' --seed 1 --latent_act exp --comment exp --init_scale 0.0001 && 
./fit_vae.sh '3' 'MNIST' 'gaussian' 'lin|lin' --seed 2 --latent_act exp --comment exp --init_scale 0.0001 && 
./fit_vae.sh '3' 'MNIST' 'gaussian' 'mlp+b|lin' --seed 2 --latent_act exp --comment exp --init_scale 0.0001 && 
./fit_vae.sh '3' 'MNIST' 'gaussian' 'lin|lin' --seed 3 --latent_act exp --comment exp --init_scale 0.0001 && 
./fit_vae.sh '3' 'MNIST' 'gaussian' 'mlp+b|lin' --seed 3 --latent_act exp --comment exp --init_scale 0.0001 && 
./fit_vae.sh '3' 'MNIST' 'gaussian' 'lin|lin' --seed 4 --latent_act exp --comment exp --init_scale 0.0001 && 
./fit_vae.sh '3' 'MNIST' 'gaussian' 'mlp+b|lin' --seed 4 --latent_act exp --comment exp --init_scale 0.0001 && 
./fit_vae.sh '3' 'MNIST' 'gaussian' 'lin|lin' --seed 5 --latent_act exp --comment exp --init_scale 0.0001 && 
./fit_vae.sh '3' 'MNIST' 'gaussian' 'mlp+b|lin' --seed 5 --latent_act exp --comment exp --init_scale 0.0001

In [14]:
print(scripts_mach)

{
    0: [
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'lin|lin' --seed 1 --method mc",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'lin|lin' --seed 1 --method exact",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'mlp+b|lin' --seed 1 --method mc",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'mlp+b|lin' --seed 1 --method exact",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'lin|lin' --seed 2 --method mc",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'lin|lin' --seed 2 --method exact",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'mlp+b|lin' --seed 2 --method mc",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'mlp+b|lin' --seed 2 --method exact",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'lin|lin' --seed 3 --method mc",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'lin|lin' --seed 3 --method exact",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'mlp+b|lin' --seed 3 --method mc",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'mlp+b|lin' --seed 3 --method exact",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'lin|lin' --seed 4 --method mc",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'lin|lin' --seed 4 --method exact",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'mlp+b|lin' --seed 4 --method mc",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'mlp+b|lin' --seed 4 --method exact",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'lin|lin' --seed 5 --method mc",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'lin|lin' --seed 5 --method exact",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'mlp+b|lin' --seed 5 --method mc",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'mlp+b|lin' --seed 5 --method exact",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'lin|lin' --seed 1 --method mc",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'lin|lin' --seed 1 --method exact",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'mlp+b|lin' --seed 1 --method mc",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'mlp+b|lin' --seed 1 --method exact",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'lin|lin' --seed 2 --method mc",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'lin|lin' --seed 2 --method exact",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'mlp+b|lin' --seed 2 --method mc",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'mlp+b|lin' --seed 2 --method exact",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'lin|lin' --seed 3 --method mc",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'lin|lin' --seed 3 --method exact",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'mlp+b|lin' --seed 3 --method mc",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'mlp+b|lin' --seed 3 --method exact",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'lin|lin' --seed 4 --method mc",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'lin|lin' --seed 4 --method exact",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'mlp+b|lin' --seed 4 --method mc",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'mlp+b|lin' --seed 4 --method exact",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'lin|lin' --seed 5 --method mc",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'lin|lin' --seed 5 --method exact",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'mlp+b|lin' --seed 5 --method mc",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'mlp+b|lin' --seed 5 --method exact",
        "./fit_vae.sh '0' 'MNIST' 'poisson' 'lin|lin' --seed 1 --method mc",
        "./fit_vae.sh '0' 'MNIST' 'poisson' 'lin|lin' --seed 1 --method exact",
        "./fit_vae.sh '0' 'MNIST' 'poisson' 'mlp+b|lin' --seed 1 --method mc",
        "./fit_vae.sh '0' 'MNIST' 'poisson' 'mlp+b|lin' --seed 1 --method exact",
        "./fit_vae.sh '0' 'MNIST' 'poisson' 'lin|lin' --seed 2 --method mc",
        "./fit_vae.sh '0' 'MNIST' 'poisson' 'lin|lin' --seed 2 --method exact",
        "./fit_vae.sh '0' 'MNIST' 'poisson' 'mlp+b|lin' --seed 2 --method mc",
        "./fit_vae.sh '0' 'MNIST' 'poisson' 'mlp+b|lin'

In [15]:
print(scripts_divided)

[
    [
        "./fit_vae.sh '3' 'DOVES' 'laplace' 'lin|lin' --seed 1 --method mc",
        "./fit_vae.sh '3' 'DOVES' 'laplace' 'lin|lin' --seed 1 --method exact",
        "./fit_vae.sh '3' 'DOVES' 'laplace' 'mlp+b|lin' --seed 1 --method mc",
        "./fit_vae.sh '3' 'DOVES' 'laplace' 'mlp+b|lin' --seed 1 --method exact",
        "./fit_vae.sh '3' 'DOVES' 'laplace' 'lin|lin' --seed 2 --method mc",
        "./fit_vae.sh '3' 'DOVES' 'laplace' 'lin|lin' --seed 2 --method exact",
        "./fit_vae.sh '3' 'DOVES' 'laplace' 'mlp+b|lin' --seed 2 --method mc",
        "./fit_vae.sh '3' 'DOVES' 'laplace' 'mlp+b|lin' --seed 2 --method exact",
        "./fit_vae.sh '3' 'DOVES' 'laplace' 'lin|lin' --seed 3 --method mc",
        "./fit_vae.sh '3' 'DOVES' 'laplace' 'lin|lin' --seed 3 --method exact",
        "./fit_vae.sh '3' 'DOVES' 'laplace' 'mlp+b|lin' --seed 3 --method mc",
        "./fit_vae.sh '3' 'DOVES' 'laplace' 'mlp+b|lin' --seed 3 --method exact",
        "./fit_vae.sh '3' 'DOVES' 'laplace' 'lin|lin' --seed 4 --method mc",
        "./fit_vae.sh '3' 'DOVES' 'laplace' 'lin|lin' --seed 4 --method exact",
        "./fit_vae.sh '3' 'DOVES' 'laplace' 'mlp+b|lin' --seed 4 --method mc"
    ],
    [
        "./fit_vae.sh '3' 'DOVES' 'laplace' 'mlp+b|lin' --seed 4 --method exact",
        "./fit_vae.sh '3' 'DOVES' 'laplace' 'lin|lin' --seed 5 --method mc",
        "./fit_vae.sh '3' 'DOVES' 'laplace' 'lin|lin' --seed 5 --method exact",
        "./fit_vae.sh '3' 'DOVES' 'laplace' 'mlp+b|lin' --seed 5 --method mc",
        "./fit_vae.sh '3' 'DOVES' 'laplace' 'mlp+b|lin' --seed 5 --method exact",
        "./fit_vae.sh '3' 'CIFAR10-PATCHES' 'laplace' 'lin|lin' --seed 1 --method mc",
        "./fit_vae.sh '3' 'CIFAR10-PATCHES' 'laplace' 'lin|lin' --seed 1 --method exact",
        "./fit_vae.sh '3' 'CIFAR10-PATCHES' 'laplace' 'mlp+b|lin' --seed 1 --method mc",
        "./fit_vae.sh '3' 'CIFAR10-PATCHES' 'laplace' 'mlp+b|lin' --seed 1 --method exact",
        "./fit_vae.sh '3' 'CIFAR10-PATCHES' 'laplace' 'lin|lin' --seed 2 --method mc",
        "./fit_vae.sh '3' 'CIFAR10-PATCHES' 'laplace' 'lin|lin' --seed 2 --method exact",
        "./fit_vae.sh '3' 'CIFAR10-PATCHES' 'laplace' 'mlp+b|lin' --seed 2 --method mc",
        "./fit_vae.sh '3' 'CIFAR10-PATCHES' 'laplace' 'mlp+b|lin' --seed 2 --method exact",
        "./fit_vae.sh '3' 'CIFAR10-PATCHES' 'laplace' 'lin|lin' --seed 3 --method mc",
        "./fit_vae.sh '3' 'CIFAR10-PATCHES' 'laplace' 'lin|lin' --seed 3 --method exact"
    ],
    [
        "./fit_vae.sh '3' 'CIFAR10-PATCHES' 'laplace' 'mlp+b|lin' --seed 3 --method mc",
        "./fit_vae.sh '3' 'CIFAR10-PATCHES' 'laplace' 'mlp+b|lin' --seed 3 --method exact",
        "./fit_vae.sh '3' 'CIFAR10-PATCHES' 'laplace' 'lin|lin' --seed 4 --method mc",
        "./fit_vae.sh '3' 'CIFAR10-PATCHES' 'laplace' 'lin|lin' --seed 4 --method exact",
        "./fit_vae.sh '3' 'CIFAR10-PATCHES' 'laplace' 'mlp+b|lin' --seed 4 --method mc",
        "./fit_vae.sh '3' 'CIFAR10-PATCHES' 'laplace' 'mlp+b|lin' --seed 4 --method exact",
        "./fit_vae.sh '3' 'CIFAR10-PATCHES' 'laplace' 'lin|lin' --seed 5 --method mc",
        "./fit_vae.sh '3' 'CIFAR10-PATCHES' 'laplace' 'lin|lin' --seed 5 --method exact",
        "./fit_vae.sh '3' 'CIFAR10-PATCHES' 'laplace' 'mlp+b|lin' --seed 5 --method mc",
        "./fit_vae.sh '3' 'CIFAR10-PATCHES' 'laplace' 'mlp+b|lin' --seed 5 --method exact",
        "./fit_vae.sh '3' 'MNIST' 'laplace' 'lin|lin' --seed 1 --method mc",
        "./fit_vae.sh '3' 'MNIST' 'laplace' 'lin|lin' --seed 1 --method exact",
        "./fit_vae.sh '3' 'MNIST' 'laplace' 'mlp+b|lin' --seed 1 --method mc",
        "./fit_vae.sh '3' 'MNIST' 'laplace' 'mlp+b|lin' --seed 1 --method exact",
        "./fit_vae.sh '3' 'MNIST' 'laplace' 'lin|lin' --seed 2 --method mc"
    ],
    [
        "./fit_vae.sh '3' 'MNIST' 'laplace' 'lin|lin' --seed 2 --method exact",
        "./fit_vae.sh '3' 'MNIST' 'laplace' 'mlp+b|lin' --seed 2 --method mc",
        "./fit_vae.sh 

## Table 1 (solo)

All conv models

In [16]:
host = 'solo'
_cleanup(save_dir, host)

scripts_solo = collections.defaultdict(list)
tot = 0

gpu_mapping = {
    0: 0,
    1: 0,
    2: 1,
    3: 1,
}

### Part 1: mc vs. exact (main)

In [17]:
n_seeds = 5
seeds = range(1, n_seeds + 1)
args = ['--method mc', '--method exact']
datasets = ['DOVES', 'CIFAR10-PATCHES', 'MNIST']
model_types = ['poisson', 'categorical', 'gaussian', 'laplace']
archi_solo = ['conv+b|lin']

In [18]:
combos_outer = itertools.product(
    enumerate(model_types), datasets)
for (gpu_i, m), d in combos_outer:
    combos_inner = itertools.product(
        seeds, archi_solo, args)
    for s, a, arg in combos_inner:
        kws = dict(
            device=gpu_mapping[gpu_i],
            dataset=d,
            model=m,
            archi=a,
            seed=s,
            args=arg,
        )
        scripts_solo[gpu_mapping[gpu_i]].append(job_runner_script(**kws))
        tot += 1

In [19]:
print(tot)

120

### Part 2: gaussian + act fn

In [20]:
activations = ['relu', 'softplus', 'square', 'exp']

In [21]:
combos_outer = itertools.product(
    enumerate(activations), datasets)
for (gpu_i, fn), d in combos_outer:
    # arg
    arg = f"--latent_act {fn} --comment {fn}"
    if fn == 'exp':
        arg = f"{arg} --init_scale 0.0001"

    combos_inner = itertools.product(
        seeds, archi_solo)
    for s, a in combos_inner:
        kws = dict(
            device=gpu_mapping[gpu_i],
            dataset=d,
            archi=a,
            seed=s,
            args=arg,
            model='gaussian',
        )
        scripts_solo[gpu_mapping[gpu_i]].append(job_runner_script(**kws))
        tot += 1

In [22]:
print(tot)

180

In [23]:
scripts_solo = dict(scripts_solo)
print({k: len(v) for k, v in scripts_solo.items()})

{0: 90, 1: 90}

### Save

In [24]:
n_fits = 5

for gpu_i, scripts in scripts_solo.items():
    scripts_divided = divide_list(scripts, n_fits)
    for fit_i, s in enumerate(scripts_divided):
        combined = ' && '.join(s)
        save_obj(
            obj=combined,
            file_name=_name(host, gpu_i, fit_i),
            save_dir=save_dir,
            mode='txt',
        )

[PROGRESS] 'solo-cuda0-fit0.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'solo-cuda0-fit1.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'solo-cuda0-fit2.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'solo-cuda0-fit3.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'solo-cuda0-fit4.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'solo-cuda1-fit0.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'solo-cuda1-fit1.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'solo-cuda1-fit2.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'solo-cuda1-fit3.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'solo-cuda1-fit4.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

### Print one to check

In [25]:
print(combined.replace('&& ', '&& \n'))

./fit_vae.sh '1' 'MNIST' 'gaussian' 'conv+b|lin' --seed 3 --latent_act square --comment square && 
./fit_vae.sh '1' 'MNIST' 'gaussian' 'conv+b|lin' --seed 4 --latent_act square --comment square && 
./fit_vae.sh '1' 'MNIST' 'gaussian' 'conv+b|lin' --seed 5 --latent_act square --comment square && 
./fit_vae.sh '1' 'DOVES' 'gaussian' 'conv+b|lin' --seed 1 --latent_act exp --comment exp --init_scale 0.0001 && 
./fit_vae.sh '1' 'DOVES' 'gaussian' 'conv+b|lin' --seed 2 --latent_act exp --comment exp --init_scale 0.0001 && 
./fit_vae.sh '1' 'DOVES' 'gaussian' 'conv+b|lin' --seed 3 --latent_act exp --comment exp --init_scale 0.0001 && 
./fit_vae.sh '1' 'DOVES' 'gaussian' 'conv+b|lin' --seed 4 --latent_act exp --comment exp --init_scale 0.0001 && 
./fit_vae.sh '1' 'DOVES' 'gaussian' 'conv+b|lin' --seed 5 --latent_act exp --comment exp --init_scale 0.0001 && 
./fit_vae.sh '1' 'CIFAR10-PATCHES' 'gaussian' 'conv+b|lin' --seed 1 --latent_act exp --comment exp --init_scale 
0.0001 && 
./fit_vae.sh '1' 'CIFAR10-PATCHES' 'gaussian' 'conv+b|lin' --seed 2 --latent_act exp --comment exp --init_scale 
0.0001 && 
./fit_vae.sh '1' 'CIFAR10-PATCHES' 'gaussian' 'conv+b|lin' --seed 3 --latent_act exp --comment exp --init_scale 
0.0001 && 
./fit_vae.sh '1' 'CIFAR10-PATCHES' 'gaussian' 'conv+b|lin' --seed 4 --latent_act exp --comment exp --init_scale 
0.0001 && 
./fit_vae.sh '1' 'CIFAR10-PATCHES' 'gaussian' 'conv+b|lin' --seed 5 --latent_act exp --comment exp --init_scale 
0.0001 && 
./fit_vae.sh '1' 'MNIST' 'gaussian' 'conv+b|lin' --seed 1 --latent_act exp --comment exp --init_scale 0.0001 && 
./fit_vae.sh '1' 'MNIST' 'gaussian' 'conv+b|lin' --seed 2 --latent_act exp --comment exp --init_scale 0.0001 && 
./fit_vae.sh '1' 'MNIST' 'gaussian' 'conv+b|lin' --seed 3 --latent_act exp --comment exp --init_scale 0.0001 && 
./fit_vae.sh '1' 'MNIST' 'gaussian' 'conv+b|lin' --seed 4 --latent_act exp --comment exp --init_scale 0.0001 && 
./fit_vae.sh '1' 'MNIST' 'gaussian' 'conv+b|lin' --seed 5 --latent_act exp --comment exp --init_scale 0.0001

In [26]:
print(scripts_solo)

{
    0: [
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'conv+b|lin' --seed 1 --method mc",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'conv+b|lin' --seed 1 --method exact",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'conv+b|lin' --seed 2 --method mc",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'conv+b|lin' --seed 2 --method exact",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'conv+b|lin' --seed 3 --method mc",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'conv+b|lin' --seed 3 --method exact",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'conv+b|lin' --seed 4 --method mc",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'conv+b|lin' --seed 4 --method exact",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'conv+b|lin' --seed 5 --method mc",
        "./fit_vae.sh '0' 'DOVES' 'poisson' 'conv+b|lin' --seed 5 --method exact",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'conv+b|lin' --seed 1 --method mc",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'conv+b|lin' --seed 1 --method exact",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'conv+b|lin' --seed 2 --method mc",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'conv+b|lin' --seed 2 --method exact",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'conv+b|lin' --seed 3 --method mc",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'conv+b|lin' --seed 3 --method exact",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'conv+b|lin' --seed 4 --method mc",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'conv+b|lin' --seed 4 --method exact",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'conv+b|lin' --seed 5 --method mc",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'poisson' 'conv+b|lin' --seed 5 --method exact",
        "./fit_vae.sh '0' 'MNIST' 'poisson' 'conv+b|lin' --seed 1 --method mc",
        "./fit_vae.sh '0' 'MNIST' 'poisson' 'conv+b|lin' --seed 1 --method exact",
        "./fit_vae.sh '0' 'MNIST' 'poisson' 'conv+b|lin' --seed 2 --method mc",
        "./fit_vae.sh '0' 'MNIST' 'poisson' 'conv+b|lin' --seed 2 --method exact",
        "./fit_vae.sh '0' 'MNIST' 'poisson' 'conv+b|lin' --seed 3 --method mc",
        "./fit_vae.sh '0' 'MNIST' 'poisson' 'conv+b|lin' --seed 3 --method exact",
        "./fit_vae.sh '0' 'MNIST' 'poisson' 'conv+b|lin' --seed 4 --method mc",
        "./fit_vae.sh '0' 'MNIST' 'poisson' 'conv+b|lin' --seed 4 --method exact",
        "./fit_vae.sh '0' 'MNIST' 'poisson' 'conv+b|lin' --seed 5 --method mc",
        "./fit_vae.sh '0' 'MNIST' 'poisson' 'conv+b|lin' --seed 5 --method exact",
        "./fit_vae.sh '0' 'DOVES' 'categorical' 'conv+b|lin' --seed 1 --method mc",
        "./fit_vae.sh '0' 'DOVES' 'categorical' 'conv+b|lin' --seed 1 --method exact",
        "./fit_vae.sh '0' 'DOVES' 'categorical' 'conv+b|lin' --seed 2 --method mc",
        "./fit_vae.sh '0' 'DOVES' 'categorical' 'conv+b|lin' --seed 2 --method exact",
        "./fit_vae.sh '0' 'DOVES' 'categorical' 'conv+b|lin' --seed 3 --method mc",
        "./fit_vae.sh '0' 'DOVES' 'categorical' 'conv+b|lin' --seed 3 --method exact",
        "./fit_vae.sh '0' 'DOVES' 'categorical' 'conv+b|lin' --seed 4 --method mc",
        "./fit_vae.sh '0' 'DOVES' 'categorical' 'conv+b|lin' --seed 4 --method exact",
        "./fit_vae.sh '0' 'DOVES' 'categorical' 'conv+b|lin' --seed 5 --method mc",
        "./fit_vae.sh '0' 'DOVES' 'categorical' 'conv+b|lin' --seed 5 --method exact",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'categorical' 'conv+b|lin' --seed 1 --method mc",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'categorical' 'conv+b|lin' --seed 1 --method exact",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'categorical' 'conv+b|lin' --seed 2 --method mc",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'categorical' 'conv+b|lin' --seed 2 --method exact",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'categorical' 'conv+b|lin' --seed 3 --method mc",
        "./fit_vae.sh '0' 'CIFAR10-PATCHES' 'categorical' 'conv+b|lin' --seed 3 --method exact",
        "./fit_v

In [27]:
print(scripts_divided)

[
    [
        "./fit_vae.sh '1' 'DOVES' 'gaussian' 'conv+b|lin' --seed 1 --method mc",
        "./fit_vae.sh '1' 'DOVES' 'gaussian' 'conv+b|lin' --seed 1 --method exact",
        "./fit_vae.sh '1' 'DOVES' 'gaussian' 'conv+b|lin' --seed 2 --method mc",
        "./fit_vae.sh '1' 'DOVES' 'gaussian' 'conv+b|lin' --seed 2 --method exact",
        "./fit_vae.sh '1' 'DOVES' 'gaussian' 'conv+b|lin' --seed 3 --method mc",
        "./fit_vae.sh '1' 'DOVES' 'gaussian' 'conv+b|lin' --seed 3 --method exact",
        "./fit_vae.sh '1' 'DOVES' 'gaussian' 'conv+b|lin' --seed 4 --method mc",
        "./fit_vae.sh '1' 'DOVES' 'gaussian' 'conv+b|lin' --seed 4 --method exact",
        "./fit_vae.sh '1' 'DOVES' 'gaussian' 'conv+b|lin' --seed 5 --method mc",
        "./fit_vae.sh '1' 'DOVES' 'gaussian' 'conv+b|lin' --seed 5 --method exact",
        "./fit_vae.sh '1' 'CIFAR10-PATCHES' 'gaussian' 'conv+b|lin' --seed 1 --method mc",
        "./fit_vae.sh '1' 'CIFAR10-PATCHES' 'gaussian' 'conv+b|lin' --seed 1 --method exact",
        "./fit_vae.sh '1' 'CIFAR10-PATCHES' 'gaussian' 'conv+b|lin' --seed 2 --method mc",
        "./fit_vae.sh '1' 'CIFAR10-PATCHES' 'gaussian' 'conv+b|lin' --seed 2 --method exact",
        "./fit_vae.sh '1' 'CIFAR10-PATCHES' 'gaussian' 'conv+b|lin' --seed 3 --method mc",
        "./fit_vae.sh '1' 'CIFAR10-PATCHES' 'gaussian' 'conv+b|lin' --seed 3 --method exact",
        "./fit_vae.sh '1' 'CIFAR10-PATCHES' 'gaussian' 'conv+b|lin' --seed 4 --method mc",
        "./fit_vae.sh '1' 'CIFAR10-PATCHES' 'gaussian' 'conv+b|lin' --seed 4 --method exact"
    ],
    [
        "./fit_vae.sh '1' 'CIFAR10-PATCHES' 'gaussian' 'conv+b|lin' --seed 5 --method mc",
        "./fit_vae.sh '1' 'CIFAR10-PATCHES' 'gaussian' 'conv+b|lin' --seed 5 --method exact",
        "./fit_vae.sh '1' 'MNIST' 'gaussian' 'conv+b|lin' --seed 1 --method mc",
        "./fit_vae.sh '1' 'MNIST' 'gaussian' 'conv+b|lin' --seed 1 --method exact",
        "./fit_vae.sh '1' 'MNIST' 'gaussian' 'conv+b|lin' --seed 2 --method mc",
        "./fit_vae.sh '1' 'MNIST' 'gaussian' 'conv+b|lin' --seed 2 --method exact",
        "./fit_vae.sh '1' 'MNIST' 'gaussian' 'conv+b|lin' --seed 3 --method mc",
        "./fit_vae.sh '1' 'MNIST' 'gaussian' 'conv+b|lin' --seed 3 --method exact",
        "./fit_vae.sh '1' 'MNIST' 'gaussian' 'conv+b|lin' --seed 4 --method mc",
        "./fit_vae.sh '1' 'MNIST' 'gaussian' 'conv+b|lin' --seed 4 --method exact",
        "./fit_vae.sh '1' 'MNIST' 'gaussian' 'conv+b|lin' --seed 5 --method mc",
        "./fit_vae.sh '1' 'MNIST' 'gaussian' 'conv+b|lin' --seed 5 --method exact",
        "./fit_vae.sh '1' 'DOVES' 'laplace' 'conv+b|lin' --seed 1 --method mc",
        "./fit_vae.sh '1' 'DOVES' 'laplace' 'conv+b|lin' --seed 1 --method exact",
        "./fit_vae.sh '1' 'DOVES' 'laplace' 'conv+b|lin' --seed 2 --method mc",
        "./fit_vae.sh '1' 'DOVES' 'laplace' 'conv+b|lin' --seed 2 --method exact",
        "./fit_vae.sh '1' 'DOVES' 'laplace' 'conv+b|lin' --seed 3 --method mc",
        "./fit_vae.sh '1' 'DOVES' 'laplace' 'conv+b|lin' --seed 3 --method exact"
    ],
    [
        "./fit_vae.sh '1' 'DOVES' 'laplace' 'conv+b|lin' --seed 4 --method mc",
        "./fit_vae.sh '1' 'DOVES' 'laplace' 'conv+b|lin' --seed 4 --method exact",
        "./fit_vae.sh '1' 'DOVES' 'laplace' 'conv+b|lin' --seed 5 --method mc",
        "./fit_vae.sh '1' 'DOVES' 'laplace' 'conv+b|lin' --seed 5 --method exact",
        "./fit_vae.sh '1' 'CIFAR10-PATCHES' 'laplace' 'conv+b|lin' --seed 1 --method mc",
        "./fit_vae.sh '1' 'CIFAR10-PATCHES' 'laplace' 'conv+b|lin' --seed 1 --method exact",
        "./fit_vae.sh '1' 'CIFAR10-PATCHES' 'laplace' 'conv+b|lin' --seed 2 --method mc",
        "./fit_vae.sh '1' 'CIFAR10-PATCHES' 'laplace' 'conv+b|lin' --seed 2 --method exact",
        "./fit_vae.sh '1' 'CIFAR10-PATCHES' 'laplace' 'conv+b|lin' --seed 3 --method mc",
        "./fit_vae.sh '1' 'CIFAR10-PATCHES' 'laplace' 'conv+b|lin' --seed 3 --method exact",
        "./fit_vae.sh '1' 'CI

## Machine: ```chewie```

Task: st (prepared for Dekel)

In [5]:
host = 'chewie'

args = ['--comment st']
model_types = ['poisson', 'categorical']
archi = ['lin|lin', 'mlp+b|lin', 'conv+b|lin']

In [6]:
tot = 0

combos_outer = itertools.product(
    enumerate(model_types),
    enumerate(datasets),
)
for (gpu_i, m), (fit_i, d) in combos_outer:
    combos_inner = itertools.product(
        range(1, n_seeds + 1),
        archi,
        args,
    )
    script = []
    for s, a, arg in combos_inner:
        script.append(job_runner_script(
            device=gpu_i,
            dataset=d,
            model=m,
            archi=a,
            seed=s,
            args=arg,
        ))
    tot += len(script)
    script = ' && '.join(script)
    # save
    save_obj(
        obj=script,
        file_name=_name(host, gpu_i, fit_i),
        save_dir=save_dir,
        mode='txt',
    )

[PROGRESS] 'chewie-cuda0-fit0.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'chewie-cuda0-fit1.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'chewie-cuda0-fit2.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'chewie-cuda1-fit0.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'chewie-cuda1-fit1.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'chewie-cuda1-fit2.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

In [7]:
print(f"tot #fits: {tot}")

tot #fits: 90

In [8]:
print(script.replace('&& ', '&& \n'))

./fit_vae.sh '1' 'MNIST' 'categorical' 'lin|lin' --seed 1 --comment st && 
./fit_vae.sh '1' 'MNIST' 'categorical' 'mlp+b|lin' --seed 1 --comment st && 
./fit_vae.sh '1' 'MNIST' 'categorical' 'conv+b|lin' --seed 1 --comment st && 
./fit_vae.sh '1' 'MNIST' 'categorical' 'lin|lin' --seed 2 --comment st && 
./fit_vae.sh '1' 'MNIST' 'categorical' 'mlp+b|lin' --seed 2 --comment st && 
./fit_vae.sh '1' 'MNIST' 'categorical' 'conv+b|lin' --seed 2 --comment st && 
./fit_vae.sh '1' 'MNIST' 'categorical' 'lin|lin' --seed 3 --comment st && 
./fit_vae.sh '1' 'MNIST' 'categorical' 'mlp+b|lin' --seed 3 --comment st && 
./fit_vae.sh '1' 'MNIST' 'categorical' 'conv+b|lin' --seed 3 --comment st && 
./fit_vae.sh '1' 'MNIST' 'categorical' 'lin|lin' --seed 4 --comment st && 
./fit_vae.sh '1' 'MNIST' 'categorical' 'mlp+b|lin' --seed 4 --comment st && 
./fit_vae.sh '1' 'MNIST' 'categorical' 'conv+b|lin' --seed 4 --comment st && 
./fit_vae.sh '1' 'MNIST' 'categorical' 'lin|lin' --seed 5 --comment st && 
./fit_vae.sh '1' 'MNIST' 'categorical' 'mlp+b|lin' --seed 5 --comment st && 
./fit_vae.sh '1' 'MNIST' 'categorical' 'conv+b|lin' --seed 5 --comment st